# Linear Optimization Assignment 2

## Group Members
- Tanmay Garg CS20BTECH11063
- Aayush Patel CS20BTECH11001
- Ganesh Bombatkar CS20BTECH11016
- Kartik Srinivas ES20BTECH11015

In [1]:
import numpy as np
from numpy import linalg as la
import csv

In [2]:
class SimplexAlgorithm():
    def __init__(self, A, b, c, x = None):
        self.A = A
        self.b = b
        self.m, self.n = A.shape
        if x is None:
            # self.x = np.empty([n, 1]) # n,
            pass
        else:
            self.x = x # n,
        self.c = c

        self.eps = 1e-6

        assert(self.A.shape == (self.m, self.n))

        assert(self.b.shape == (self.m, ))
        assert(self.b.ndim == 1)

        assert(self.c.shape == (self.n, ))
        assert(self.c.ndim == 1)

        assert(self.x.shape == (self.n, ))

    def is_feasiability(self, point):
        return np.all(self.A @ point <= self.b)
        
    def is_vertex(self, point):
        return len(np.where(np.abs(self.A @ point - self.b) <= self.eps)[0]) == self.n

    def get_direction(self):
        tight_rows = np.where(np.abs(self.A @ self.x - self.b) <= self.eps)[0]
        # shape of tight_rows: (n, ) since x is a vertex of a non-degenerate polytope)
        assert len(tight_rows) == self.n

        A_tight = self.A[tight_rows]
        # Shape of A_tight: (n, n)
        assert A_tight.shape == (self.n, self.n)

        tight_rows_bool = np.zeros(self.m, dtype=bool) # (m, )
        tight_rows_bool[tight_rows] = True
        untight_rows = np.where(tight_rows_bool == False)[0] # (m - n, )
        assert untight_rows.shape == (self.m - self.n, )

        A_untight = self.A[untight_rows]
        assert A_untight.shape == (self.m - self.n, self.n)

        b_untight = self.b[untight_rows]
        assert b_untight.shape == (self.m - self.n, )

        negative_indices = np.where(la.inv(A_tight.T) @ self.c < 0)[0]
        if len(negative_indices) != 0:
            negative_index = negative_indices[0]
            v = -la.inv(A_tight.T)[negative_index] # (n, )

            if len(np.where(self.A @ v > 0)[0]) == 0:
                # The linear program is unbounded
                return None, 1
            
            t = (b_untight - np.dot(A_untight, self.x)) / (np.dot(A_untight, v) + 1e-12)
            t = t[t >= 0]
            min_t = np.min(t)
            return min_t * v, 0
        else:
            return None, 0
        
    def get_vertex_from_feasible_point(self):
        return self.x

    def solve(self):
        # Check if the initial point is feasible
        if not self.is_feasiability(self.x):
            print("The initial point is not feasible")
            return
        
        self.x = self.get_vertex_from_feasible_point()

        if not self.is_vertex(self.x):
            print("Point x is not a vertex of the polytope")
            return

        print(f"The initial point is {self.x} with cost {self.c.T @ self.x}")
        
        # Moving from the initial point to a vertex
        iter_count = 0
        while True:
            dir, status = self.get_direction()
            if dir is None:
                if status == 1:
                    print("The linear program is unbounded")
                    return
                elif status == 0:
                    # self.x is the optimal vertex
                    break
            else:
                self.x = self.x + dir
                print(f"At iteration {iter_count + 1}, the current point is {self.x} with cost {self.c.T @ self.x}")
                iter_count += 1

        print(f"The optimal point is {self.x} with cost {self.c.T @ self.x}")

In [3]:
# c = np.array([4, 1], dtype=np.float32)
# x = np.array([0, 50], dtype=np.float32)
# b = np.array([50, 90, 0, 0], dtype=np.float32)
# A = np.array([[1, 1], [3, 1], [-1, 0], [0, -1]], dtype=np.float32)

def read_input_from_csv(file_path):
    with open(file_path, 'r') as csv_file:
        reader = csv.reader(csv_file)
        data = list(reader)

    # Extracting data from CSV
    x = np.array(data[0][:-1], dtype=np.float32)
    c = np.array(data[1][:-1], dtype=np.float32)
    b = np.array(data[2:], dtype=np.float32)[:, -1]
    A = np.array([list(map(float, row[:-1])) for row in data[2:]], dtype=np.float32)

    return x, c, b, A

In [4]:
file_path = '../Test/test1.csv'  
x, c, b, A = read_input_from_csv(file_path)

algo = SimplexAlgorithm(A, b, c, x)
algo.solve()

The initial point is [0. 0.] with cost 0.0
At iteration 1, the current point is [30.  0.] with cost 120.0
The optimal point is [30.  0.] with cost 120.0
